**Querying NCBI genome sequences**

Here you can find Python scripts for querying NCBI genome promoter sequences.

Questions and suggestions should be sent to abraao@neuro.ufrn.br


---------------------------------------------------------------------------------


Author: Abraão LP Andrade

In [1]:
import pandas as pd
import numpy as np
import Bio as bio
import os

from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [2]:
print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("bio:", bio.__version__)

pandas: 1.0.5
numpy: 1.17.5
bio: 1.78


Chromossome sequences (GenBank sequence): https://www.ncbi.nlm.nih.gov/assembly/GCF_003957565.1/
 - folder_path//chromossome_seq//files


Gene coordinate tables (Download>Track Data): https://www.ncbi.nlm.nih.gov/genome/gdv/browser/genome/?id=GCF_003957565.1 
 - folder_path//gene_table//files

In [3]:
print("FOLDER PATHS ------------------------------------------------------")
folder_path = input("enter the general folder path:               ") # D:/TCC/DADOS/%NCBI genoma/bTaeGut1/
chromossome_seq_folder_name = input("enter the chromossome sequence folder name:  ") # chromossome_seq
gene_table_folder_name = input("enter the gene coordinate table folder name: ") # gene_table
destination_folder_name = input("enter the destination query folder name:     ") # promoter_seqs

print("\nPROMOTER LIMITS ---------------------------------------------------")
up_limit = int(input("enter the upstream limit:   "))
down_limit = int(input("enter the downstream limit: "))

FOLDER PATHS ------------------------------------------------------
enter the general folder path:               D:/TCC/DADOS/%NCBI genoma/bTaeGut1/
enter the chromossome sequence folder name:  chromossome_seq
enter the gene coordinate table folder name: gene_table
enter the destination query folder name:     promoter_seqs

PROMOTER LIMITS ---------------------------------------------------
enter the upstream limit:   -1000
enter the downstream limit: +500


In [5]:
os.chdir(folder_path) # moving to the main directory
filename_chromseq_list = os.listdir(path = chromossome_seq_folder_name) # chromosome sequence file name list

filename_chromseq_list

['chrom_10_seq.fasta',
 'chrom_11_seq.fasta',
 'chrom_12_seq.fasta',
 'chrom_13_seq.fasta',
 'chrom_14_seq.fasta',
 'chrom_15_seq.fasta',
 'chrom_16_seq.fasta',
 'chrom_17_seq.fasta',
 'chrom_18_seq.fasta',
 'chrom_19_seq.fasta',
 'chrom_1A_seq.fasta',
 'chrom_1_seq.fasta',
 'chrom_20_seq.fasta',
 'chrom_21_seq.fasta',
 'chrom_22_seq.fasta',
 'chrom_23_seq.fasta',
 'chrom_24_seq.fasta',
 'chrom_25_seq.fasta',
 'chrom_26_seq.fasta',
 'chrom_27_seq.fasta',
 'chrom_28_seq.fasta',
 'chrom_29_seq.fasta',
 'chrom_2_seq.fasta',
 'chrom_30_seq.fasta',
 'chrom_3_seq.fasta',
 'chrom_4A_seq.fasta',
 'chrom_4_seq.fasta',
 'chrom_5_seq.fasta',
 'chrom_6_seq.fasta',
 'chrom_7_seq.fasta',
 'chrom_8_seq.fasta',
 'chrom_9_seq.fasta',
 'chrom_mt_seq.fasta',
 'chrom_NW_022045285_seq.fasta',
 'chrom_NW_022045286_seq.fasta',
 'chrom_NW_022045287_seq.fasta',
 'chrom_NW_022045288_seq.fasta',
 'chrom_NW_022045289_seq.fasta',
 'chrom_NW_022045290_seq.fasta',
 'chrom_NW_022045291_seq.fasta',
 'chrom_NW_02204529

In [6]:
os.makedirs(destination_folder_name)
    
evaluating_results = {} 
for file_name in filename_chromseq_list:
    PROMOTER_SEQS = []
    
    CHROM_SEQ = list(SeqIO.parse(chromossome_seq_folder_name + "/" + file_name, "fasta"))
    REVERSE_CHROM_SEQ = CHROM_SEQ[0].reverse_complement()
    
    chrom_name = file_name[:-10]
    REFSEQ_CHROM = pd.read_csv(gene_table_folder_name + "/" + chrom_name + "_gene_table.CSV", skiprows = 1)
    print(chrom_name)
    
    
    for gene_idx in range(len(REFSEQ_CHROM)):
        
        
        # 1. SELECTING THE PROMOTER REGION (up/down) ---------------------------------------
        coord = np.array([REFSEQ_CHROM["Start"][gene_idx], REFSEQ_CHROM["Stop"][gene_idx]], dtype = int)

            # 1.1. checking the strand
        if REFSEQ_CHROM["Strand"][gene_idx] == "plus": 
            tss = coord[0]
            # 1.2. if the upstream region is smaller   
            # than 1000 select the existing region
            if tss>1000:
                gene_seq = CHROM_SEQ[0].seq[tss+up_limit-1:tss+down_limit-1]
            else: 
                gene_seq = CHROM_SEQ[0].seq[0:tss+down_limit-1]
        else:
            tss = len(CHROM_SEQ[0].seq)-coord[1]
            if tss>1000:
                gene_seq = REVERSE_CHROM_SEQ.seq[tss+up_limit:tss+down_limit]
            else:
                gene_seq = REVERSE_CHROM_SEQ.seq[0:tss+down_limit]
        # -------------------------------------------------------------------------------------
          
            
        # 2. WRITING THE FASTA DESCRIPTION ----------------------------------------------------
            # GeneID|GeneSymbol|Chromossome|Start|Stop|Strand|Accession 
        description = "{}|{}|{}|{}|{}|{}|{}".format(REFSEQ_CHROM["NCBI Gene ID"][gene_idx], REFSEQ_CHROM["Gene symbol"][gene_idx], 
                                                 chrom_name, coord[0], coord[1], REFSEQ_CHROM["Strand"][gene_idx],
                                                 REFSEQ_CHROM["Accession"][gene_idx])
        # ------------------------------------------------------------------------------------
        
        
        # 3. CREATING RECORDER AND ADDING TO A LIST ------------------------------------------
        new_record = SeqRecord(gene_seq, description, "", "")
        PROMOTER_SEQS.append(new_record)
        # ------------------------------------------------------------------------------------
        
    
    # 4. WRITING THE PROMOTER SEQUENCES FILE -------------------------------------------------    
    SeqIO.write(PROMOTER_SEQS, r"{}\Chrom_{}.fasta".format(destination_folder_name, chrom_name), "fasta")
    # ----------------------------------------------------------------------------------------

chrom_10
chrom_11
chrom_12
chrom_13
chrom_14
chrom_15
chrom_16
chrom_17
chrom_18
chrom_19
chrom_1A
chrom_1
chrom_20
chrom_21
chrom_22
chrom_23
chrom_24
chrom_25
chrom_26
chrom_27
chrom_28
chrom_29
chrom_2
chrom_30
chrom_3
chrom_4A
chrom_4
chrom_5
chrom_6
chrom_7
chrom_8
chrom_9
chrom_mt
chrom_NW_022045285
chrom_NW_022045286
chrom_NW_022045287
chrom_NW_022045288
chrom_NW_022045289
chrom_NW_022045290
chrom_NW_022045291
chrom_NW_022045292
chrom_NW_022045293
chrom_NW_022045294
chrom_NW_022045295
chrom_NW_022045296
chrom_NW_022045297
chrom_NW_022045298
chrom_NW_022045299
chrom_NW_022045300
chrom_NW_022045301
chrom_NW_022045302
chrom_NW_022045304
chrom_NW_022045305
chrom_NW_022045306
chrom_NW_022045307
chrom_NW_022045308
chrom_NW_022045309
chrom_NW_022045310
chrom_NW_022045311
chrom_NW_022045312
chrom_NW_022045313
chrom_NW_022045314
chrom_NW_022045315
chrom_NW_022045316
chrom_NW_022045317
chrom_NW_022045318
chrom_NW_022045319
chrom_NW_022045320
chrom_NW_022045321
chrom_NW_022045322
chrom_NW_